In [9]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:
spark = SparkSession.builder.appName("Question2").getOrCreate()

In [5]:
df = spark.read.csv('Sales Data.csv', header=True)

In [6]:
df.show()

+---+--------+--------------------+----------------+----------+----------------+--------------------+-----+------+--------------+----+
|_c0|Order ID|             Product|Quantity Ordered|Price Each|      Order Date|    Purchase Address|Month| Sales|          City|Hour|
+---+--------+--------------------+----------------+----------+----------------+--------------------+-----+------+--------------+----+
|  0|  295665|  Macbook Pro Laptop|               1|      1700|30-12-2019 00:01|136 Church St, Ne...|   12|  1700| New York City|   0|
|  1|  295666|  LG Washing Machine|               1|       600|29-12-2019 07:03|562 2nd St, New Y...|   12|   600| New York City|   7|
|  2|  295667|USB-C Charging Cable|               1|     11.95|12-12-2019 18:21|277 Main St, New ...|   12| 11.95| New York City|  18|
|  3|  295668|    27in FHD Monitor|               1|    149.99|22-12-2019 15:13|410 6th St, San F...|   12|149.99| San Francisco|  15|
|  4|  295669|USB-C Charging Cable|               1|   

In [11]:
for column in df.columns:
  if df.filter(col(column).isNull()).count() > 0:
    print(column)

Sales


In [15]:
column_name = 'Sales' # Replace with your column name
# Calculate the mean for the column
mean_value = df.select(mean(column_name)).collect()[0][0]
# Fill missing values in the specific column with the calculated mean
df = df.fillna({column_name: mean_value})

In [16]:
df=df.dropDuplicates()

In [17]:
df = df.withColumn("Quantity Ordered", col("Quantity Ordered").cast("Integer"))
df = df.withColumn("Price Each", col("Price Each").cast("Float"))
df = df.withColumn("Sales", col("Sales").cast("Float"))

In [18]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: integer (nullable = true)
 |-- Price Each: float (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Sales: float (nullable = true)
 |-- City: string (nullable = true)
 |-- Hour: string (nullable = true)



In [20]:
df = df.filter(
(col("Sales") >= 0) &
(col("Price Each") >= 0) &
(col("Quantity Ordered") >= 0)
)

In [21]:
df_total_sales = df.groupBy("Product").agg(
sum("Sales").alias("TotalSales")
)
# Show the result
df_total_sales.show()

+--------------------+------------------+
|             Product|        TotalSales|
+--------------------+------------------+
|    Wired Headphones|246651.92929840088|
|  Macbook Pro Laptop|         8037600.0|
|Apple Airpods Hea...|         2349150.0|
|              iPhone|         4794300.0|
|Lightning Chargin...|347094.14547920227|
|Bose SoundSport H...|1345565.4012680054|
|USB-C Charging Cable| 286674.7890357971|
|AAA Batteries (4-...| 92740.83064889908|
|        20in Monitor| 454148.7011795044|
|    27in FHD Monitor|1132424.5414733887|
|     Vareebadd Phone|          827200.0|
|34in Ultrawide Mo...|2355557.9494628906|
|            LG Dryer|          387600.0|
|AA Batteries (4-p...|106300.05223083496|
|        Google Phone|         3319200.0|
|       Flatscreen TV|         1445700.0|
|  LG Washing Machine|          399600.0|
|27in 4K Gaming Mo...|2435097.4990234375|
|     ThinkPad Laptop|4129958.6596679688|
+--------------------+------------------+

